# 0. Import libraries and packages

In [ ]:
import pandas as pd
import os

# 1. Datasets - Load and Exploration

## 1.1 Fact table - Casinodaily 

In [ ]:
casinodaily = pd.read_csv('../datasets/casinodaily.csv')

In [ ]:
casinodaily.info()

In [ ]:
casinodaily.head(20)

In [ ]:
casinodaily.CurrencyId.unique()

In [ ]:
casinodaily[casinodaily['CurrencyId']==15]

In [ ]:
casinodaily[casinodaily['CurrencyId']==15].count()

In [ ]:
casinodaily[casinodaily['CurrencyId']==10].CountryId.unique()

## 1.2 Dimension table - Users

In [ ]:
users = pd.read_csv('../datasets/users.csv')

In [ ]:
users.info()

In [ ]:
users.head()

In [ ]:
users.Country.unique()

### Convert BirthDate datatype to date

In [ ]:
users['BirthDate'] =  pd.to_datetime(users['BirthDate'], errors = 'coerce')

In [ ]:
users.info()

## 1.3 Dimension table - Casino Manufacturers

In [ ]:
manufacturers = pd.read_csv('../datasets/casinomanufacturers.csv', sep = ', "', engine = 'python')

In [ ]:
manufacturers.info()

In [ ]:
manufacturers.head(30)

### Split/spread data into multiple columns

In [ ]:
manufacturers['"CasinoManufacturerId,""CasinoManufacturerName"",""FromDate"",""ToDate"",""LatestFlag"""'].str.split(',',n=4,expand=True).set_axis(['CasinoManufacturerId','CasinoManufacturerName','FromDate', 'ToDate', 'LatestFlag'],axis=1)

In [ ]:
manufacturers = manufacturers['"CasinoManufacturerId,""CasinoManufacturerName"",""FromDate"",""ToDate"",""LatestFlag"""'].str.split(',',n=4,expand=True).set_axis(['CasinoManufacturerId','CasinoManufacturerName','FromDate', 'ToDate', 'LatestFlag'],axis=1)

### Remove all " from column values

In [ ]:
manufacturers['CasinoManufacturerId'] = manufacturers['CasinoManufacturerId'].str.replace('"', '')
manufacturers['CasinoManufacturerName'] = manufacturers['CasinoManufacturerName'].str.replace('"', '')
manufacturers['LatestFlag'] = manufacturers['LatestFlag'].str.replace('"', '')

### Convert CasinoManufacturerId and LatestFlag columns to integer

In [ ]:
manufacturers[['CasinoManufacturerId', 'LatestFlag']] = manufacturers[['CasinoManufacturerId', 'LatestFlag']].apply(pd.to_numeric)

In [ ]:
manufacturers[manufacturers['LatestFlag'] == 1]

In [ ]:
manufacturers.info()

### Keeping only the latest manufacturers names

In [ ]:
manufacturers = manufacturers[manufacturers['LatestFlag'] == 1]

## 1.4 Dimension table - Casino Providers

In [ ]:
providers = pd.read_csv('../datasets/casinoproviders.csv')

In [ ]:
providers.info()

In [ ]:
providers.head(50)

### Dropping 4 rows. 2 of them have same id and similar name (EGTGaming, ESAGaming), the other 2 have same id and different name.

In [ ]:
providers.drop([2, 10, 25, 29], inplace= True)

## 1.5 Dimension table - Currency Rates

In [ ]:
currencyrates = pd.read_csv('../datasets/currencyrates.csv')

In [ ]:
currencyrates.info()

In [ ]:
currencyrates

In [ ]:
currencyrates.ToCurrencyId.unique()

# 3. Joined data table

In [ ]:
casinodaily = casinodaily [casinodaily['CurrencyId']==15]

I have excluded data with CurrencyId = 10 since the Currency Rate table has only currency rate for CurrencyId = 15.

In [ ]:
final_table = casinodaily \
    .merge(users, how = 'left', left_on = 'UserID', right_on= 'user_id')\
    .merge(manufacturers, how = 'left', on = 'CasinoManufacturerId')\
    .merge(providers, how = 'left', on = 'CasinoProviderId')\
    .merge(currencyrates, how = 'left', left_on =['CurrencyId', 'Date'], right_on = ['ToCurrencyId', 'Date'])

In [ ]:
final_table.info()

### Create Age column to create later AgeGroup column  
Instead of now any particular date can be used to do the following calculations

In [ ]:
now = pd.to_datetime('now')

In [ ]:
final_table['Age']=(now - final_table['BirthDate']).dt.total_seconds() / (60*60*24*365.25)

### Create AgeGroup column

In [ ]:
final_table['AgeGroup'] = pd.cut(final_table['Age'], 
                            [0, 18, 21, 27, 33, 41, 51, 150], 
                            labels=['Under 18', '18-20', '21-26', '27-32', '33-40', '41-50', '50+'])

### Currency conversions

In [ ]:
final_table['GGR_EUR'] = final_table['GGR']*final_table['EuroRate']
final_table['Returns_EUR'] = final_table['Returns']*final_table['EuroRate']

In [ ]:
final_table.columns

In [ ]:
final_table = final_table[['Date', 'Country', 'Sex', 'AgeGroup','VIPStatus', 'CasinoManufacturerName', 'CasinoProviderName', 'GGR', 'Returns', 'GGR_EUR', 'Returns_EUR']]

In [ ]:
final_table.head(50)

In [ ]:
final_table.count()

# 4. Aggregated Data

In [ ]:
agr = final_table.groupby(['Date', 'Country', 'Sex', 'AgeGroup', 'VIPStatus', 'CasinoManufacturerName', 'CasinoProviderName'], as_index = False)[['GGR', 'Returns', 'GGR_EUR', 'Returns_EUR']].agg('sum')

In [ ]:
agr = agr[agr['Returns']!=0].reset_index()

In [ ]:
agr.columns

In [ ]:
agr.drop(['index'], axis=1, inplace= True)

In [ ]:
agr.columns

In [ ]:
agr

### Create target subdirectory

In [ ]:
os.makedirs('../target', exist_ok=True)  

### Write data to csv file in target subdirectory

In [ ]:
agr.to_csv('../target/aggregated_data.csv') 